In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

In [12]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')


#工事費用
cost_df = DataFrame(pd.read_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/prefecture_cost.csv', encoding='shift_jis'))
pref = ['北海道',
        '青森',
        '岩手',
        '宮城',
        '秋田',
        '山形',
        '福島',
        '茨城',
        '栃木',
        '群馬',
        '埼玉',
        '千葉',
        '東京都',
        '神奈川',
        '新潟',
        '富山',
        '石川',
        '福井',
        '山梨',
        '長野',
        '岐阜',
        '静岡',
        '愛知',
        '三重',
        '滋賀',
        '京都',
        '大阪',
        '兵庫',
        '奈良',
        '和歌山',
        '鳥取',
        '島根',
        '岡山',
        '広島',
        '山口',
        '徳島',
        '香川',
        '愛媛',
        '高知',
        '福岡',
        '佐賀',
        '長崎',
        '熊本',
        '大分',
        '宮崎',
        '鹿児島',
        '沖繩']
pref_df = DataFrame(pref, columns=['prefecture'])
cost_df = pd.merge(pref_df, cost_df, on='prefecture')
#損害額の計算
def dmg_value(cost, total_area, dpc_rate, age, dmg_rate):
        #建物の金額
        house_price = cost * total_area
        #償却された金額
        dpc_price = house_price * 0.9 * dpc_rate * age
        #時価総額
        tmp_price = house_price - dpc_price
        #築年が多いため時価総額がマイナスになった場合の条件分岐
        if (tmp_price / house_price) < 0.05:
                tmp_price = house_price * 0.05
        #損害額
        dmg = tmp_price * dmg_rate
        return dmg

n = int(start_pref)
while n <= int(finish_pref):
    df = pd.read_csv(f'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data//{n}_step8_1.csv')
    dmg_price_list = []

    #工事費用
    df['cost'] = np.where(df['estimated_structure'] == 0, cost_df['wood'][n] * 1000, cost_df['steel'][n] * 1000)
    #償却率
    df['dpc_rate'] = np.where(df['estimated_structure'] == 0, 0.031, 0.036)
    #築年数
    df['age'] = df['estimated_age'].map({1: 5, 2: 8, 3: 15.5, 4: 25.5, 5: 35.5, 6: 45.5, 7: 50})
    
    df['dmg_rate'] = np.where((df['A31_001'] == 11) | (df['A31_001'] == 21), np.where(df['floor'] == 0, 0.4, 0.35),
                          np.where((df['A31_001'] == 12) | (df['A31_001'] == 22), np.where(df['floor'] == 0, 0.6, 0.45),
                          np.where((df['A31_001'] == 13) | (df['A31_001'] == 23), np.where(df['floor'] == 0, 0.75, 0.5),
                          np.where(((df['A31_001'].between(14, 15)) | (df['A31_001'] >= 24)), np.where(df['floor'] == 0, 0.8, 0.55), 0.0))))
    
    df['damage_price'] = df.apply(lambda row: dmg_value(row['cost'], row['total_area'], row['dpc_rate'], row['age'], row['dmg_rate']), axis=1)
#     df = df.join(DataFrame(dmg_price_list, columns=['damage_price']))
    
    #df.to_csv(f'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/output/{n}_step8_1_output.csv', index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 1
終了する都道府県コードを入力してください
終了する都道府県コー : 1


C:\Users\pipoz\AppData\Local\Temp\ipykernel_18056\3242619255.py:78: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data//{n}_step8_1.csv')


In [5]:
df.groupby('KEY_CODE', as_index=False)['damage_price'].sum()

,KEY_CODE,damage_price
0,141010180,9.604426e+08
1,141010490,2.556908e+09
2,141010560,1.623599e+09
3,141010570,1.237905e+09
4,141010580,1.910518e+09
...,...,...
1360,14366009001,2.109556e+08
1361,14366009002,5.511168e+05
1362,14366009003,9.168066e+07
1363,14366009004,9.471514e+07


In [13]:
df['KEY_CODE'].value_counts().sum()

265446

In [13]:
df

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,damage_price
0,2510,141.296158,43.056621,0,143.515,1364,143.515,1101120312,1,6,21,256000,0.036,45.5,0.40,7.347968e+05
1,2527,141.295720,43.056429,0,151.005,1364,151.005,1101120312,1,4,21,256000,0.036,25.5,0.40,2.687454e+06
2,2528,141.295604,43.056521,0,133.200,1364,133.200,1101120312,1,4,21,256000,0.036,25.5,0.40,2.370576e+06
3,2532,141.295842,43.056316,0,99.710,1364,99.710,1101120312,1,4,21,256000,0.036,25.5,0.40,1.774551e+06
4,2555,141.295215,43.056908,0,322.025,1364,322.025,,1,6,21,256000,0.036,45.5,0.40,1.648768e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249508,12081,141.731384,43.280488,0,91.630,1364,91.630,12100910,1,4,26,256000,0.036,25.5,0.80,3.261500e+06
249509,5024,141.675608,43.234963,0,142.320,1364,142.320,12100860,1,2,23,256000,0.036,8.0,0.75,2.024269e+07
249510,5029,141.675463,43.238065,0,145.060,1364,145.060,12100850,1,3,23,256000,0.036,15.5,0.75,1.386449e+07
249511,5039,141.675453,43.242219,0,92.395,1364,92.395,12100850,1,1,22,256000,0.036,5.0,0.60,1.189279e+07
